# Problem Statement

- A local healthcare company published multiple articles containing healthcare facts,information, and tips. It wishes to create a conversational chatbot that can address readers’concerns in natural language using information from the trusted articles and in the healthcare context.
- The conversational chatbot should answer readers' queries using only the information from the published articles. Where appropriate, it should adopt an empathetic and understanding
tone.

# Solution

In [1]:
!apt-get -qq install poppler-utils tesseract-ocr
%pip install -q --user --upgrade pillow
%pip install -q unstructured["all-docs"]==0.12.5
!pip install openai langchain langchain-openai

Selecting previously unselected package poppler-utils.
(Reading database ... 121752 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.3_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.3) ...
Selecting previously unselected package tesseract-ocr-eng.
Preparing to unpack .../tesseract-ocr-eng_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-eng (1:4.00~git30-7274cfa-1.1) ...
Selecting previously unselected package tesseract-ocr-osd.
Preparing to unpack .../tesseract-ocr-osd_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-osd (1:4.00~git30-7274cfa-1.1) ...
Selecting previously unselected package tesseract-ocr.
Preparing to unpack .../tesseract-ocr_4.1.1-2.1build1_amd64.deb ...
Unpacking tesseract-ocr (4.1.1-2.1build1) ...
Setting up tesseract-ocr-eng (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-osd (1:4.00~git30-7274cfa-1.1) ...
Setting up poppler-utils (22.02.0-2ubuntu0.3) ...
Setting up tess

In [2]:
import os

os.environ["OPENAI_API_KEY"] = ""

Retrieve PDF and HTML files

In [3]:
import requests
from pprint import pprint

download_folder = "/content/data"

urls = \
[
    "https://www.ace-hta.gov.sg/docs/default-source/default-library/managing-pre-diabetes-(updated-on-27-jul-2021)c2bfc77474154c2abf623156a4b93002.pdf",
    "https://www.ace-hta.gov.sg/docs/default-source/acgs/acg-t2dm-personalising-medications.pdf",
    "https://www.ace-hta.gov.sg/docs/default-source/acgs/gdm---an-update-on-screening-diagnosis-and-follow-up-(may-2018).pdf",
    "https://ch-api.healthhub.sg/api/public/content/8fcd2bf22c2f472c8efbe5c650e3969e?v=a8fc8264",
    "https://www.healthhub.sg/a-z/diseases-and-conditions/diabetes-treatment-capsules--tablets",
    "https://www.healthhub.sg/a-z/diseases-and-conditions/diabetic_foot_ttsh",
    "https://www.healthhub.sg/a-z/diseases-and-conditions/diabetic-ulcer",
    "https://www.healthhub.sg/a-z/diseases-and-conditions/diabetes-treatment-insulin",
    "https://www.healthxchange.sg/diabetes/living-well-diabetes/diabetes-recommended-vaccinations-children-adults",
    "https://ch-api.healthhub.sg/api/public/content/57349b96ccfe47319fb49d902a064022?v=6a601253&_gl=1*ok09oc*_ga*MTk3Mjg0Mjc5Ni4xNzA3Nzg5OTUw*_ga_VQW1KL2RMR*MTcxMDc1MzAxMS40LjEuMTcxMDc1NDQxNC4xMi4wLjA.",
    "https://ch-api.healthhub.sg/api/public/content/70e56617c5b140139bb5cb37db3cdef0?v=a7dd48f7&3_gl=1*11zfakr*_ga*MTk3Mjg0Mjc5Ni4xNzA3Nzg5OTUw*_ga_VQW1KL2RMR*MTcxMDc1MzAxMS40LjEuMTcxMDc1NDQzMC42MC4wLjA.",
    "https://ch-api.healthhub.sg/api/public/content/0aa3838b8eeb40e6b34a2340a65fdd03?v=eca50be0&_gl=1*1tq2sup*_ga*MTk3Mjg0Mjc5Ni4xNzA3Nzg5OTUw*_ga_VQW1KL2RMR*MTcxMDc1MzAxMS40LjEuMTcxMDc1NDQzMC42MC4wLjA.",
    "https://ch-api.healthhub.sg/api/public/content/57c95e67895840888240d7322f00117c?v=f00ca9b4&_gl=1*1tq2sup*_ga*MTk3Mjg0Mjc5Ni4xNzA3Nzg5OTUw"
]

if not os.path.exists(download_folder):
  os.makedirs(download_folder)

_ = [os.remove(os.path.join(download_folder, fname)) for fname in os.listdir(download_folder)]

for index, url in enumerate(urls):
  try:
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
    response = requests.get(url, headers = headers)

    content_type = response.headers.get('Content-Type')

    if "pdf" in content_type:
      file_name = str(index) + "_" + "pdf" + "_" + url.split("/")[-1]

      if not file_name.endswith(".pdf"):
        file_name += ".pdf"

    elif "html" in content_type:
      file_name = str(index) + "_" + "html" + "_" + url.split("/")[-1] + ".html"

    file_path = os.path.join(download_folder, file_name)

    with open(file_path, "wb") as file:
      file.write(response.content)

    print(f"Downloaded {file_name} successfully.\n")

  except Exception as e:
    print(f"error occurred for {index}_{file_name}: {e}")

Downloaded 0_pdf_managing-pre-diabetes-(updated-on-27-jul-2021)c2bfc77474154c2abf623156a4b93002.pdf successfully.

Downloaded 1_pdf_acg-t2dm-personalising-medications.pdf successfully.

Downloaded 2_pdf_gdm---an-update-on-screening-diagnosis-and-follow-up-(may-2018).pdf successfully.

Downloaded 3_pdf_8fcd2bf22c2f472c8efbe5c650e3969e?v=a8fc8264.pdf successfully.

Downloaded 4_html_diabetes-treatment-capsules--tablets.html successfully.

Downloaded 5_html_diabetic_foot_ttsh.html successfully.

Downloaded 6_html_diabetic-ulcer.html successfully.

Downloaded 7_html_diabetes-treatment-insulin.html successfully.

Downloaded 8_html_diabetes-recommended-vaccinations-children-adults.html successfully.

Downloaded 9_pdf_57349b96ccfe47319fb49d902a064022?v=6a601253&_gl=1*ok09oc*_ga*MTk3Mjg0Mjc5Ni4xNzA3Nzg5OTUw*_ga_VQW1KL2RMR*MTcxMDc1MzAxMS40LjEuMTcxMDc1NDQxNC4xMi4wLjA..pdf successfully.

Downloaded 10_pdf_70e56617c5b140139bb5cb37db3cdef0?v=a7dd48f7&3_gl=1*11zfakr*_ga*MTk3Mjg0Mjc5Ni4xNzA3Nzg5OTUw*

Helper functions to extract text and table from PDFs and HTMLs using Unstructured framework

In [4]:
from unstructured.partition.pdf import partition_pdf
from unstructured.partition.auto import partition_html
from unstructured.cleaners.core import replace_unicode_quotes, bytes_string_to_string, clean, clean_non_ascii_chars
from unstructured.chunking.title import chunk_by_title

import pandas as pd

def extract_text_elements_pdf(pdf_fpath):
  elements = partition_pdf(filename=pdf_fpath, strategy = "hi_res", infer_table_structure=True)

  df_text_elements = pd.DataFrame({"sno": [], "parent_id": [], "depth": [], "label": [], "text": [], "page_num": [], "is_continuation": []})

  for index, el in enumerate(elements):
    el.apply(replace_unicode_quotes)

    text = el.text
    text = clean(text, extra_whitespace=True, dashes = True, bullets = True)

    label = el.category

    if label == "Table":
      continue

    row = {"sno": int(index), "parent_id": el.metadata.parent_id, "depth": el.metadata.category_depth, "label": label, "text": text, "page_num": el.metadata.page_number, "is_continuation": el.metadata.is_continuation}
    row_df = pd.DataFrame([row])

    df_text_elements = pd.concat([df_text_elements, row_df], ignore_index = True)

  text_chunks = chunk_by_title(elements)

  return df_text_elements, text_chunks

def extract_table_elements_pdf(pdf_fpath):
  elements = partition_pdf(filename=pdf_fpath, strategy = "hi_res", infer_table_structure=True)

  table_elements = [el.metadata.text_as_html for el in elements if el.category == "Table"]

  return table_elements

def extract_text_elements_html(html_fpath):
  elements = partition_html(filename = html_fpath)

  df_text_elements = pd.DataFrame({"sno": [], "parent_id": [], "depth": [], "label": [], "text": [], "page_num": [], "is_continuation": []})

  for index, el in enumerate(elements):
    el.apply(replace_unicode_quotes)

    text = el.text
    text = clean(text, extra_whitespace=True, dashes = True, bullets = True)

    label = el.category

    row = {"sno": int(index), "parent_id": el.metadata.parent_id, "depth": el.metadata.category_depth, "label": label, "text": text, "page_num": el.metadata.page_number, "is_continuation": el.metadata.is_continuation}
    row_df = pd.DataFrame([row])

    df_text_elements = pd.concat([df_text_elements, row_df], ignore_index = True)

  text_chunks = chunk_by_title(elements)

  return df_text_elements, text_chunks

Rule based semantic chunking of overlapping window size of N

In [5]:
def get_new_chunks(chunks, n = 2):
  text_chunks_new = []

  for i in range(len(chunks) - 1):
    curr_chunks = chunks[i:i+n]

    new_chunk = ""
    for chunk in curr_chunks:
      new_chunk += chunk.text + "\n\n"
    text_chunks_new.append(new_chunk)

  return text_chunks_new

LLM summarization of extracted table

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

model = ChatOpenAI(temperature = 0, model = "gpt-3.5-turbo")

prompt_text = """You are an assistant tasked with summarizing tables and text. Give a concise summary of the table or text. The table is formatted in HTML: {element}"""
prompt = ChatPromptTemplate.from_template(prompt_text)

summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

def extract_table_summaries(table_elements):
  table_summaries = summarize_chain.batch(table_elements, {"max_concurrency": 5})

  return table_summaries

Extract text and table from PDFs and HTMLs

In [7]:
texts = []
metadatas = []
fname_list = []
urls_list = []

fnames = os.listdir(download_folder)

for fname in fnames:
  idx = int(fname.split("_")[0])
  url = urls[idx]

  print(f"> Processing for URL: {url}")
  fpath = os.path.join(download_folder, fname)

  if fpath.endswith(".pdf"):
    df_text_elements, text_chunks = extract_text_elements_pdf(fpath)
    text_chunks_new = get_new_chunks(text_chunks)

    table_elements = extract_table_elements_pdf(fpath)

    table_summaries = []

    if len(table_elements) > 0:
      table_summaries = extract_table_summaries(table_elements)

    if len(table_summaries) > 0:
      for table_summary in table_summaries:
        text_chunks_new.append(table_summary)

  elif fpath.endswith(".html"):
    df_text_elements, text_chunks = extract_text_elements_html(fpath)
    text_chunks_new = get_new_chunks(text_chunks)

  for index, text_chunk in enumerate(text_chunks_new):
    text = text_chunk
    metadata = {"source": url}

    texts.append(text)
    metadatas.append(metadata)
    fname_list.append(fname)
    urls_list.append(url)

docs_df = pd.DataFrame({"text": texts,"source": fname_list, "urls": urls_list, "metadata": metadatas})
docs_df.to_pickle("/content/docs_df.pkl")

display(docs_df.head(10))
display(pd.DataFrame(docs_df["urls"].value_counts()))

> Processing for URL: https://ch-api.healthhub.sg/api/public/content/0aa3838b8eeb40e6b34a2340a65fdd03?v=eca50be0&_gl=1*1tq2sup*_ga*MTk3Mjg0Mjc5Ni4xNzA3Nzg5OTUw*_ga_VQW1KL2RMR*MTcxMDc1MzAxMS40LjEuMTcxMDc1NDQzMC42MC4wLjA.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

...

> Processing for URL: https://www.healthhub.sg/a-z/diseases-and-conditions/diabetes-treatment-insulin
> Processing for URL: https://ch-api.healthhub.sg/api/public/content/70e56617c5b140139bb5cb37db3cdef0?v=a7dd48f7&3_gl=1*11zfakr*_ga*MTk3Mjg0Mjc5Ni4xNzA3Nzg5OTUw*_ga_VQW1KL2RMR*MTcxMDc1MzAxMS40LjEuMTcxMDc1NDQzMC42MC4wLjA.


> Processing for URL: https://www.ace-hta.gov.sg/docs/default-source/acgs/acg-t2dm-personalising-medications.pdf


> Processing for URL: https://www.healthhub.sg/a-z/diseases-and-conditions/diabetes-treatment-capsules--tablets


,text,source,urls,metadata
0,Se BE PROACTIVE. —\n\nBE PROACTIVE\n\nSetting ...,11_pdf_0aa3838b8eeb40e6b34a2340a65fdd03?v=eca5...,https://ch-api.healthhub.sg/api/public/content...,{'source': 'https://ch-api.healthhub.sg/api/pu...
1,Helpful ways to improve sleep quality\n\nMaint...,11_pdf_0aa3838b8eeb40e6b34a2340a65fdd03?v=eca5...,https://ch-api.healthhub.sg/api/public/content...,{'source': 'https://ch-api.healthhub.sg/api/pu...
2,Consider different areas of your life and whic...,11_pdf_0aa3838b8eeb40e6b34a2340a65fdd03?v=eca5...,https://ch-api.healthhub.sg/api/public/content...,{'source': 'https://ch-api.healthhub.sg/api/pu...
3,Setting my goal\n\nWhere I want to be: Manage ...,11_pdf_0aa3838b8eeb40e6b34a2340a65fdd03?v=eca5...,https://ch-api.healthhub.sg/api/public/content...,{'source': 'https://ch-api.healthhub.sg/api/pu...
4,What might get in the way What I can do Person...,11_pdf_0aa3838b8eeb40e6b34a2340a65fdd03?v=eca5...,https://ch-api.healthhub.sg/api/public/content...,{'source': 'https://ch-api.healthhub.sg/api/pu...
5,\n\n6\n\n• Complete the “Setting my goal” hand...,11_pdf_0aa3838b8eeb40e6b34a2340a65fdd03?v=eca5...,https://ch-api.healthhub.sg/api/public/content...,{'source': 'https://ch-api.healthhub.sg/api/pu...
6,6\n\n• Complete the “Setting my goal” handout ...,11_pdf_0aa3838b8eeb40e6b34a2340a65fdd03?v=eca5...,https://ch-api.healthhub.sg/api/public/content...,{'source': 'https://ch-api.healthhub.sg/api/pu...
7,0 Not at all important\n\nDo you think you can...,11_pdf_0aa3838b8eeb40e6b34a2340a65fdd03?v=eca5...,https://ch-api.healthhub.sg/api/public/content...,{'source': 'https://ch-api.healthhub.sg/api/pu...
8,"Do you think you can reach this goal?\n\n0 No,...",11_pdf_0aa3838b8eeb40e6b34a2340a65fdd03?v=eca5...,https://ch-api.healthhub.sg/api/public/content...,{'source': 'https://ch-api.healthhub.sg/api/pu...
9,"0 No, I cannot\n\n7\n\nStep 1: Write down the ...",11_pdf_0aa3838b8eeb40e6b34a2340a65fdd03?v=eca5...,https://ch-api.healthhub.sg/api/public/content...,{'source': 'https://ch-api.healthhub.sg/api/pu...


,count
urls,
https://ch-api.healthhub.sg/api/public/content/70e56617c5b140139bb5cb37db3cdef0?v=a7dd48f7&3_gl=1*11zfakr*_ga*MTk3Mjg0Mjc5Ni4xNzA3Nzg5OTUw*_ga_VQW1KL2RMR*MTcxMDc1MzAxMS40LjEuMTcxMDc1NDQzMC42MC4wLjA.,208
https://www.ace-hta.gov.sg/docs/default-source/acgs/acg-t2dm-personalising-medications.pdf,114
https://ch-api.healthhub.sg/api/public/content/0aa3838b8eeb40e6b34a2340a65fdd03?v=eca50be0&_gl=1*1tq2sup*_ga*MTk3Mjg0Mjc5Ni4xNzA3Nzg5OTUw*_ga_VQW1KL2RMR*MTcxMDc1MzAxMS40LjEuMTcxMDc1NDQzMC42MC4wLjA.,85
https://ch-api.healthhub.sg/api/public/content/57c95e67895840888240d7322f00117c?v=f00ca9b4&_gl=1*1tq2sup*_ga*MTk3Mjg0Mjc5Ni4xNzA3Nzg5OTUw,45
https://ch-api.healthhub.sg/api/public/content/57349b96ccfe47319fb49d902a064022?v=6a601253&_gl=1*ok09oc*_ga*MTk3Mjg0Mjc5Ni4xNzA3Nzg5OTUw*_ga_VQW1KL2RMR*MTcxMDc1MzAxMS40LjEuMTcxMDc1NDQxNC4xMi4wLjA.,40
https://www.ace-hta.gov.sg/docs/default-source/default-library/managing-pre-diabetes-(updated-on-27-jul-2021)c2bfc77474154c2abf623156a4b93002.pdf,35
https://www.ace-hta.gov.sg/docs/default-source/acgs/gdm---an-update-on-screening-diagnosis-and-follow-up-(may-2018).pdf,35
https://www.healthxchange.sg/diabetes/living-well-diabetes/diabetes-recommended-vaccinations-children-adults,29
https://www.healthhub.sg/a-z/diseases-and-conditions/diabetes-treatment-insulin,21
